In [ ]:
import subprocess

def get_gpu_usage():
    try:
        result = subprocess.check_output(
            ['nvidia-smi', '--query-gpu=utilization.gpu', '--format=csv,noheader,nounits'],
            encoding='utf-8'
        )
        usage = result.strip().split('\n')
        usage = [int(u) for u in usage]
        return usage  # 예: [23, 5] → GPU0: 23%, GPU1: 5%
    except Exception as e:
        print("GPU 사용량 확인 실패:", e)
        return []


In [ ]:
usage = get_gpu_usage()
for i, u in enumerate(usage):
    print(f"GPU {i}: {u}% 사용 중")


In [ ]:
from PyQt5.QtWidgets import QApplication, QWidget, QLabel, QVBoxLayout
import sys

class Example(QWidget):
    def __init__(self):
        super().__init__()
        self.initUI()

    def initUI(self):
        label = QLabel("배경색이 변경된 라벨입니다.")
        # label.setStyleSheet("background-color: lightblue;") # 간단한 색상 이름 사용
        label.setStyleSheet("background-color: #ADD8E6;") # 16진수 색상 코드 사용

        button = QPushButton("배경색이 변경된 버튼입니다.")
        button.setStyleSheet("background-color: yellow;")

        central_widget = QWidget()
        central_widget.setStyleSheet("background-color: lightgray;") # 위젯 자체의 배경색 변경

        layout = QVBoxLayout(central_widget)
        layout.addWidget(label)
        layout.addWidget(button)

        self.setCentralWidget(central_widget) # QMainWindow의 경우
        if not isinstance(self, QMainWindow): # 일반 QWidget의 경우
            self.setLayout(layout)

        self.setGeometry(300, 300, 300, 200)
        self.setWindowTitle('Change Background Color')
        self.show()

if __name__ == '__main__':
    app = QApplication(sys.argv)
    window = QMainWindow() # 또는 QWidget()
    example = Example()
    window.setCentralWidget(example) # QMainWindow의 경우
    if isinstance(window, QWidget): # 일반 QWidget의 경우
        window = example
    sys.exit(app.exec_())

In [ ]:
from PyQt5.QtWidgets import QWidget
from PyQt5.QtGui import QPainter, QPen, QPolygonF
from PyQt5.QtCore import Qt, QPointF, pyqtSignal
import datetime

class ROIEditor(QWidget):
    """
    ROI 영역을 그리고 확정하는 기능
    """
    roi_defined = pyqtSignal(list, int)  # 추가된 부분: 카메라 ID를 포함한 시그널

    def __init__(self, video_widget, cam_id):
        super().__init__(video_widget)
        self.video_widget = video_widget
        self.cam_id = cam_id  # 카메라 ID 저장
        self.points = []
        self.finished = False
        self.temp_point = None
        self.now = datetime.datetime.now()

    def mousePressEvent(self, event):
        if event.button() == Qt.LeftButton:
            # 좌클릭으로 점을 추가
            pos = event.pos()
            self.points.append(QPointF(pos))
            self.update()  # 화면 업데이트

        elif event.button() == Qt.RightButton and len(self.points) >= 3:
            # 우클릭 시 폴리곤을 확정
            self.finished = True
            self.temp_point = None
            polygon_coords = [(pt.x(), pt.y()) for pt in self.points]
            self.roi_defined.emit(polygon_coords, self.cam_id)  # 카메라 ID 포함하여 시그널 전송
            self.update()

    def mouseMoveEvent(self, event):
        if not self.finished:
            # 마우스를 이동하면서 마지막 점과 현재 점을 빨간색 선으로 그리기
            self.temp_point = event.pos()
            self.update()

    def paintEvent(self, event):
        print(__class__.__name__)
        print(f"{self.now.second}[DEBUG] paintEvent called for cam_id: {self.cam_id}, points: {self.points}" )
        painter = QPainter(self)

        # 점 찍기 (파란색) - 항상 그리기
        dot_pen = QPen(Qt.blue, 5)
        painter.setPen(dot_pen)
        for pt in self.points:
            painter.drawPoint(pt)

        # 선 그리기 (빨간색 또는 파란색)
        line_pen = QPen(Qt.red, 2)
        painter.setPen(line_pen)

        if self.points:
            polygon = QPolygonF(self.points)
            if self.finished:
                # 다각형이 완성되었으면 첫 점과 마지막 점을 연결하여 파란색으로 그린다.
                blue_pen = QPen(Qt.blue, 2)
                painter.setPen(blue_pen)
                polygon.append(self.points[0])
                painter.drawPolygon(polygon)
            else:
                # 다각형을 그리기 전에 폴리라인으로 점을 연결하고, 마우스를 이동하면서 그려지는 빨간색 선
                painter.drawPolyline(polygon)
                if self.temp_point:
                    painter.drawLine(self.points[-1], self.temp_point)

    def reset(self):
        # ROI 설정 초기화
        self.points.clear()
        self.finished = False
        self.temp_point = None
        self.update()

    def get_polygon(self):
        """[(x1, y1), (x2, y2), ...] 형식으로 반환"""
        return [(pt.x(), pt.y()) for pt in self.points] if self.finished else None

In [ ]:
from PyQt5.QtWidgets import QApplication, QWidget, QPushButton
from PyQt5.QtGui import QColor
from PyQt5.QtCore import Qt

class App(QWidget):
    def __init__(self):
        super().__init__()

        self.setWindowTitle('MN Vision 스타일')
        self.setGeometry(100, 100, 800, 600)

        # 배경색 설정
        self.setStyleSheet("background-color: #1A1A1A;")

        # 버튼 생성
        button = QPushButton('클릭하세요', self)
        button.setGeometry(300, 250, 200, 50)
        button.setStyleSheet("""
            background-color: #FF6600;
            color: #FFFFFF;
            border: none;
            border-radius: 10px;
        """)
        button.setCursor(Qt.PointingHandCursor)

        # 버튼 호버 효과
        button.setStyleSheet("""
            QPushButton {
                background-color: #FF6600;
                color: #FFFFFF;
                border: none;
                border-radius: 10px;
            }
            QPushButton:hover {
                background-color: #00B0FF;
            }
        """)

if __name__ == '__main__':
    app = QApplication([])
    window = App()
    window.show()
    app.exec_()


In [ ]:

# class VideoThread(QThread):
#     frame_ready = pyqtSignal(np.ndarray)
#     event_triggered = pyqtSignal(float, str)
#     mute_triggered = pyqtSignal(str)


#     def __init__(self, video_path, detector, postprocessor, video_buffer):
#         super().__init__()
#         self.cap = cv2.VideoCapture(video_path)
#         self.detector = detector
#         self.postprocessor = postprocessor
#         self.video_buffer = video_buffer
#         self.roi = None
#         self.running = True
#         self.frame_count = 0
        
#     # def run(self):
#     #     while self.running:
#     #         ret, frame = self.cap.read()
#     #         if not ret:
#     #             break
#     #         self.frame_count += 1
#     #         self.video_buffer.add_frame(frame.copy())
#     #         if self.frame_count % 3 != 0:
#     #             continue

#     #         results = self.postprocessor.filter_results(self.detector.detect_objects(frame))

#     #         # ROI 시각화 (파란색 선)
#     #         # if self.roi is not None:
#     #         #     cv2.polylines(frame, [self.roi], isClosed=True, color=(255, 0, 0), thickness=2)

#     #         # 객체 박스 시각화 (초록색 박스 + 라벨)
#     #         for (x1, y1, x2, y2), conf, class_name in results:
#     #             label = f"{class_name} {conf:.2f}"
#     #             cv2.rectangle(frame, (int(x1), int(y1)), (int(x2), int(y2)), (0, 255, 0), 2)
#     #             cv2.putText(frame, label, (int(x1), int(y1) - 10),
#     #                         cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 1)

#     #             # ROI 내 이벤트 트리거 체크
#     #             if self.roi is not None and cv2.pointPolygonTest(self.roi, (int(x1), int(y1)), False) >= 0:
#     #                 self.event_triggered.emit(time.time(), label)

#     #         self.frame_ready.emit(frame)
#     def run(self):
#         while self.running:
#             ret, frame = self.cap.read()
#             if not ret:
#                 break
#             self.frame_count += 1
#             self.video_buffer.add_frame(frame.copy())
#             if self.frame_count % 3 != 0:
#                 continue

#             results = self.postprocessor.filter_results(self.detector.detect_objects(frame))

#             for det in results:
#                 x1, y1, x2, y2 = det['box']
#                 conf = det['conf']
#                 class_name = det['class_name']
#                 label = f"{class_name} {conf:.2f}"

#                 # # 객체 박스 시각화 (초록색 박스 + 라벨)
#                 # cv2.rectangle(frame, (int(x1), int(y1)), (int(x2), int(y2)), (0, 255, 0), 2)
#                 # cv2.putText(frame, label, (int(x1), int(y1) - 10),
#                 #             cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 1)

#                 ## 'forklift_left' 일 경우 뮤트 트리거.
#                 ##  연결부 아직 없음. 스피커로 연결할듯?
#                 if class_name == 'forklift_left':
#                     self.mute_triggered.emit(time.time(), label)
                
#                 # 세그멘테이션 폴리곤이 있다면 시각화
#                 if det.get('polygons'):
#                     if class_name == 'person':
#                         cv2.putText(frame, label, (int(x1), int(y1) - 10),
#                                 cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 1)
#                         for poly in det['polygons']:
#                             poly_np = np.array(poly, dtype=np.int32)
#                             cv2.polylines(frame, [poly_np], isClosed=True, color=(0, 255, 0), thickness=2)
#                     else:
#                         cv2.putText(frame, label, (int(x1), int(y1) - 10),
#                                 cv2.FONT_HERSHEY_SIMPLEX, 0.5, (205, 205, 0), 1)
#                         for poly in det['polygons']:
#                             poly_np = np.array(poly, dtype=np.int32)
#                             cv2.polylines(frame, [poly_np], isClosed=True, color=(205, 205, 0), thickness=2)

#                 # ROI 내 이벤트 트리거 체크
#                 if self.roi is not None and cv2.pointPolygonTest(self.roi, (int(x1), int(y1)), False) >= 0:
#                     self.event_triggered.emit(time.time(), label)

#             self.frame_ready.emit(frame)

#     def calculate_iou(poly1, poly2):
#         poly1 = Polygon(poly1)
#         poly2 = Polygon(poly2)
#         if not poly1.is_valid or not poly2.is_valid:
#             return 0
#         intersection_area = poly1.intersection(poly2).area
#         union_area = poly1.union(poly2).area
#         if union_area == 0:
#             return 0
#         return intersection_area / union_area

#     def check_person_forklift_overlap(detections, iou_threshold=0.2):
#         """
#         person 객체와 forklift-* 객체의 polygon IoU를 계산하고 일정 임계값을 넘으면 경고 이벤트 발생
#         :param detections: [{'class_name': str, 'polygons': list[tuple], ...}, ...]
#         """
#         person_polys = [Polygon(d['polygons']) for d in detections if d['class_name'] == 'person']
#         forklift_polys = [Polygon(d['polygons']) for d in detections if d['class_name'].startswith('forklift')]

#         for person_poly in person_polys:
#             for forklift_poly in forklift_polys:
#                 if not person_poly.is_valid or not forklift_poly.is_valid:
#                     continue  # 잘못된 polygon 예외 처리

#                 inter_area = person_poly.intersection(forklift_poly).area
#                 union_area = person_poly.union(forklift_poly).area
#                 if union_area == 0:
#                     continue  # divide-by-zero 방지

#                 iou = inter_area / union_area
#                 if iou >= iou_threshold:
#                     print(f"⚠️ 위험 감지: person과 forklift 겹침 (IoU={iou:.2f})")
#                     return True  # 또는 이벤트 emit 호출

#         return False

    

#     def set_roi(self, roi_points):
#         self.roi = np.array(roi_points, dtype=np.int32)
    
#     def stop(self):
#         self.running = False
#         self.cap.release()

In [ ]:
from PyQt5.QtCore import QObject, pyqtSignal, pyqtSlot
from PyQt5.QtMultimedia import QSound
import time
import threading
import os
'./resources/etc/forklift_away.wav'
sound_path = './resources/etc/forklift_away.wav'
sound = QSound('./resources/etc/forklift_away.wav')
current_sound = sound
print("[🔈 재생 시작]")
current_sound.play()
# time.sleep(3)  # 예시: 3초 대기
print("[🧵 스레드 종료]")

In [ ]:
import sys
from PyQt5.QtWidgets import QApplication
from PyQt5.QtMultimedia import QSound
import os
app = QApplication(sys.argv)
print(os.path.abspath('./resources/etc/forklift_away.wav'))
print(os.path.exists('./resources/etc/forklift_away.wav'))


sound = QSound('./resources/etc/forklift_away.wav')
print("[🔈 재생 시작]")
sound.play()

# 이벤트 루프 실행 (소리가 재생되는 동안 유지)
# QSound는 소리가 끝날 때까지 이벤트 루프가 살아 있어야 소리가 재생됩니다.
app.exec_()


In [ ]:
import os, subprocess

def openFolder(folder_path='./resources/logs'):
    """ 
    폴더 열기 메서드
    """
    # folder_path = os.path.join(folder_path, str(1))  # 여기에 열고 싶은 폴더 경로 입력
    # folder_path = folder_path+'/1'  # 여기에 열고 싶은 폴더 경로 입력
    # folder_path = 'resources/logs/1'
    folder_path = r'C:\Users\kdt\OneDrive\바탕 화면\PROJECT_MNV\App\resources\logs\1'
    if os.path.exists(folder_path):
        subprocess.Popen(f'explorer "{folder_path}"')

        print(f"{folder_path}")
    else:
        print(f"경로가 존재하지 않습니다. {folder_path}")
# C:\Users\kdt\OneDrive\바탕 화면\PROJECT_MNV\App\resources\logs\1        
openFolder()

In [ ]:
import os

print("📁 현재 작업 디렉토리:", os.getcwd())


In [ ]:
 
    # def run(self):
    #     while self.running:
    #         ret, frame = self.cap.read()
    #         if not ret:
    #             break
    #         self.frame_count += 1
    #         self.video_buffer.add_frame(frame.copy())
    #         if self.frame_count % 2 != 0:
    #             continue

    #         results = self.postprocessor.filter_results(self.detector.detect_objects(frame))

    #         for det in results:
    #             x1, y1, x2, y2 = det['box']
    #             conf = det['conf']
    #             class_name = det['class_name']
    #             label = f"{class_name} {conf:.2f}"

    #             if class_name == 'forklift_left':
    #                 self.mute_triggered.emit(label)

    #             # 시각화
    #             if det.get('polygons'):
    #                 color = (0, 255, 0) if class_name == 'person' else (205, 205, 0)
    #                 for poly in det['polygons']:
    #                     poly_np = np.array(poly, dtype=np.int32)
    #                     cv2.polylines(frame, [poly_np], isClosed=True, color=color, thickness=2)
    #                 cv2.putText(frame, label, (int(x1), int(y1) - 10),
    #                             cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 1)
                
    #             ## 쿨타임 카운트
    #             # print(self.roi)
    #             if self.can_trigger_event():
    #                 # ▶️ person-roi 간 IoU 계산 후 이벤트 발생
    #                 person_roi_detected, person_roi_iou = self.check_person_roi_overlap(results)
                    
    #                 # iou나, pointpolytest를 통과하면 발생
    #                 if person_roi_detected or self.is_within_roi(results):
    #                     alert_manager.on_alert_signal.emit("inroi")
    #                     self.event_triggered.emit(time.time(), "person-roi overlap", person_roi_iou)

    #                 # ▶️ person-forklift 간 IoU 계산 후 이벤트 발생
    #                 overlap_detected, iou_val = self.check_person_forklift_overlap(results)
    #                 if overlap_detected :
    #                     alert_manager.on_alert_signal.emit("overlap")  # 신호(메시지)를 alert_manager에 보냄
    #                     self.event_triggered.emit(time.time(), "person-forklift overlap", iou_val)
    #         print('inroi_result',self.is_within_roi(results))
    #         self.frame_ready.emit(frame)

In [ ]:

    # def compute_polygon_iou(polygon_roi, object_box):
    #     """
    #     이거 쓰나? 
    #     polygon_roi: np.array of shape (N, 2) -> [[x1, y1], [x2, y2], ..., [xn, yn]]
    #     object_box: list or tuple -> [x1, y1, x2, y2]
    #     """
    #     roi_poly = Polygon(polygon_roi)
    #     obj_poly = box(*object_box)  # creates a rectangular polygon

    #     if not roi_poly.is_valid or not obj_poly.is_valid:
    #         return 0.0

    #     inter_area = roi_poly.intersection(obj_poly).area
    #     union_area = roi_poly.union(obj_poly).area

    #     if union_area == 0:
    #         return 0.0
    #     return inter_area / union_area            

In [1]:
import sys
from PyQt5.QtWidgets import QApplication, QWidget
from PyQt5.QtGui import QPainter, QLinearGradient, QColor, QFont, QPixmap, QPainterPath
from PyQt5.QtCore import Qt, QTimer

class GradientLabel(QWidget):
    def __init__(self, text):
        super().__init__()
        self.text = text
        self.setFont(QFont("Arial", 40, QFont.Bold))
        self.setMinimumSize(500, 150)
        self.hue = 0

        # 애니메이션 타이머
        self.timer = QTimer(self)
        self.timer.timeout.connect(self.update_gradient)
        self.timer.start(30)

    def update_gradient(self):
        self.hue = (self.hue + 2) % 360
        self.update()

    def paintEvent(self, event):
        painter = QPainter(self)
        painter.setRenderHint(QPainter.Antialiasing)

        # 텍스트 경로 생성
        path = QPainterPath()
        font = self.font()
        metrics = self.fontMetrics()
        text_width = metrics.width(self.text)
        text_height = metrics.height()

        x = (self.width() - text_width) / 2
        y = (self.height() + text_height / 2) / 2
        path.addText(x, y, font, self.text)

        # 그라데이션 생성
        gradient = QLinearGradient(0, 0, self.width(), 0)
        gradient.setColorAt(0, QColor.fromHsv(self.hue, 255, 255))
        gradient.setColorAt(1, QColor.fromHsv((self.hue + 90) % 360, 255, 255))
        painter.setBrush(gradient)
        painter.setPen(Qt.NoPen)

        # 텍스트에만 그라데이션 적용
        painter.drawPath(path)

if __name__ == '__main__':
    app = QApplication(sys.argv)
    label = GradientLabel("그라데이션 텍스트 애니메이션")
    label.show()
    sys.exit(app.exec_())


C:\Users\kdt\AppData\Local\Temp\ipykernel_27364\2206774898.py:6: DeprecationWarning: sipPyTypeDict() is deprecated, the extension module should use sipPyTypeDictRef() instead
  class GradientLabel(QWidget):


SystemExit: 0

c:\Users\kdt\anaconda3\envs\DL_TORCH\lib\site-packages\IPython\core\interactiveshell.py:3558: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [1]:
import sys
from PyQt5.QtWidgets import QWidget, QApplication
from PyQt5.QtGui import QPainter, QPainterPath, QFont, QColor, QPen
from PyQt5.QtCore import QTimer, Qt

class HoverOutlineLabel(QWidget):
    def __init__(self, text):
        super().__init__()
        self.text = text
        self.setFont(QFont("Arial", 40, QFont.Bold))
        self.setMinimumSize(500, 150)

        self.hovered = False
        self.outline_offset = 0

        self.timer = QTimer(self)
        self.timer.timeout.connect(self.update_outline)
        self.timer.start(30)

        self.setMouseTracking(True)  # 필수!

    def enterEvent(self, event):
        self.hovered = True

    def leaveEvent(self, event):
        self.hovered = False

    def update_outline(self):
        if self.hovered:
            self.outline_offset = (self.outline_offset + 1) % 10
            self.update()

    def paintEvent(self, event):
        painter = QPainter(self)
        painter.setRenderHint(QPainter.Antialiasing)

        font = self.font()
        metrics = self.fontMetrics()
        text_width = metrics.width(self.text)
        text_height = metrics.height()

        x = (self.width() - text_width) / 2
        y = (self.height() + text_height / 2) / 2

        path = QPainterPath()
        path.addText(x, y, font, self.text)

        # 기본 텍스트
        painter.setPen(Qt.NoPen)
        painter.setBrush(QColor("white"))
        painter.drawPath(path)

        # 외곽선 (호버 시만 표시)
        if self.hovered:
            pen = QPen(QColor("blue"), 2)
            pen.setDashPattern([4, 2])
            pen.setDashOffset(self.outline_offset)  # 움직이는 효과
            painter.setPen(pen)
            painter.setBrush(Qt.NoBrush)
            painter.drawPath(path)

if __name__ == '__main__':
    app = QApplication(sys.argv)
    win = HoverOutlineLabel("호버 외곽선 효과")
    win.show()
    sys.exit(app.exec_())


C:\Users\kdt\AppData\Local\Temp\ipykernel_17352\2632556810.py:6: DeprecationWarning: sipPyTypeDict() is deprecated, the extension module should use sipPyTypeDictRef() instead
  class HoverOutlineLabel(QWidget):


SystemExit: 0

c:\Users\kdt\anaconda3\envs\DL_TORCH\lib\site-packages\IPython\core\interactiveshell.py:3558: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [1]:
from PyQt5.QtWidgets import QApplication, QPushButton, QVBoxLayout, QWidget
from PyQt5.QtCore import QPropertyAnimation, QRect, QEasingCurve, QSize, Qt
from PyQt5.QtGui import QColor
import sys

class FancyButton(QPushButton):
    def __init__(self, text):
        super().__init__(text)
        self.setFixedSize(200, 80)
        
        # 기본 스타일 (그라데이션)
        self.normal_style = """
            QPushButton {
                background: qlineargradient(x1:0, y1:0, x2:1, y2:1,
                                          stop:0 #4A6CD4, stop:1 #cd74e6);
                color: white;
                border-radius: 10px;
                font-size: 16px;
                font-weight: bold;
            }
        """
        
        self.hover_style = """
            QPushButton {
                background: qlineargradient(x1:0, y1:0, x2:1, y2:1,
                                          stop:0 #5d7fe3, stop:1 #d689ec);
                color: white;
                border-radius: 10px;
                font-size: 17px;
                font-weight: bold;
            }
        """
        
        self.setStyleSheet(self.normal_style)
        
        # 크기 애니메이션
        self.size_animation = QPropertyAnimation(self, b"size")
        self.size_animation.setDuration(150)
        self.size_animation.setEasingCurve(QEasingCurve.OutQuad)
        
    def enterEvent(self, event):
        # 마우스가 올라갔을 때 크기 확대 및 스타일 변경
        self.setStyleSheet(self.hover_style)
        self.size_animation.setStartValue(self.size())
        self.size_animation.setEndValue(QSize(210, 85))
        self.size_animation.start()
        super().enterEvent(event)
        
    def leaveEvent(self, event):
        # 마우스가 벗어났을 때 크기 축소 및 스타일 복원
        self.setStyleSheet(self.normal_style)
        self.size_animation.setStartValue(self.size())
        self.size_animation.setEndValue(QSize(200, 80))
        self.size_animation.start()
        super().leaveEvent(event)
        
    def mousePressEvent(self, event):
        # 클릭 효과
        if event.button() == Qt.LeftButton:
            self.setStyleSheet("""
                QPushButton {
                    background: qlineargradient(x1:0, y1:0, x2:1, y2:1,
                                              stop:0 #3a5bb8, stop:1 #b164c9);
                    color: white;
                    border-radius: 10px;
                    font-size: 16px;
                    font-weight: bold;
                }
            """)
        super().mousePressEvent(event)
        
    def mouseReleaseEvent(self, event):
        # 클릭 후 복원
        if event.button() == Qt.LeftButton:
            self.setStyleSheet(self.normal_style)
        super().mouseReleaseEvent(event)

class Window(QWidget):
    def __init__(self):
        super().__init__()
        self.setWindowTitle("화려한 버튼")
        self.setGeometry(100, 100, 300, 200)
        
        layout = QVBoxLayout()
        self.setLayout(layout)
        
        fancy_btn = FancyButton("화려한 버튼")
        layout.addWidget(fancy_btn)

if __name__ == "__main__":
    app = QApplication(sys.argv)
    window = Window()
    window.show()
    sys.exit(app.exec_())

C:\Users\kdt\AppData\Local\Temp\ipykernel_27688\238747955.py:6: DeprecationWarning: sipPyTypeDict() is deprecated, the extension module should use sipPyTypeDictRef() instead
  class FancyButton(QPushButton):
C:\Users\kdt\AppData\Local\Temp\ipykernel_27688\238747955.py:78: DeprecationWarning: sipPyTypeDict() is deprecated, the extension module should use sipPyTypeDictRef() instead
  class Window(QWidget):


SystemExit: 0

c:\Users\kdt\anaconda3\envs\DL_TORCH\lib\site-packages\IPython\core\interactiveshell.py:3558: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [1]:
from PyQt5.QtWidgets import QApplication, QPushButton, QVBoxLayout, QWidget
from PyQt5.QtCore import QPropertyAnimation, QEasingCurve, QPoint, QTimer, Qt
from PyQt5.QtGui import QPainter, QColor, QBrush
import sys

class RippleButton(QPushButton):
    def __init__(self, text):
        super().__init__(text)
        self.setFixedSize(200, 80)
        
        # 기본 스타일
        self.setStyleSheet("""
            QPushButton {
                background-color: #4A6CD4;
                color: white;
                border-radius: 10px;
                font-size: 16px;
                font-weight: bold;
            }
        """)
        
        # 물결 효과를 위한 변수
        self.ripple_pos = QPoint()
        self.ripple_radius = 0
        self.ripple_opacity = 0
        self.ripple_animation = False
        
    def mousePressEvent(self, event):
        if event.button() == Qt.LeftButton:
            self.ripple_pos = event.pos()
            self.ripple_radius = 0
            self.ripple_opacity = 255
            self.ripple_animation = True
            
            # 타이머로 애니메이션 구현
            self.timer = QTimer(self)
            self.timer.timeout.connect(self.update_ripple)
            self.timer.start(20)  # 20ms 간격으로 업데이트
        
        super().mousePressEvent(event)
    
    def update_ripple(self):
        # 물결 반경 증가
        self.ripple_radius += 10
        # 투명도 감소
        self.ripple_opacity -= 5
        
        if self.ripple_opacity <= 0:
            self.ripple_animation = False
            self.timer.stop()
        
        self.update()  # 다시 그리기
    
    def paintEvent(self, event):
        super().paintEvent(event)
        
        # 물결 효과 그리기
        if self.ripple_animation:
            painter = QPainter(self)
            painter.setRenderHint(QPainter.Antialiasing)
            
            # 물결 색상 설정 (흰색, 투명도 조절)
            ripple_color = QColor(255, 255, 255, self.ripple_opacity)
            painter.setBrush(QBrush(ripple_color))
            painter.setPen(Qt.NoPen)
            
            # 원 그리기
            painter.drawEllipse(self.ripple_pos, self.ripple_radius, self.ripple_radius)

class Window(QWidget):
    def __init__(self):
        super().__init__()
        self.setWindowTitle("물결 효과 버튼")
        self.setGeometry(100, 100, 300, 200)
        
        layout = QVBoxLayout()
        self.setLayout(layout)
        
        ripple_btn = RippleButton("물결 효과 버튼")
        layout.addWidget(ripple_btn)

if __name__ == "__main__":
    app = QApplication(sys.argv)
    window = Window()
    window.show()
    sys.exit(app.exec_())

C:\Users\kdt\AppData\Local\Temp\ipykernel_19444\3584797096.py:6: DeprecationWarning: sipPyTypeDict() is deprecated, the extension module should use sipPyTypeDictRef() instead
  class RippleButton(QPushButton):
C:\Users\kdt\AppData\Local\Temp\ipykernel_19444\3584797096.py:70: DeprecationWarning: sipPyTypeDict() is deprecated, the extension module should use sipPyTypeDictRef() instead
  class Window(QWidget):


SystemExit: 0

c:\Users\kdt\anaconda3\envs\DL_TORCH\lib\site-packages\IPython\core\interactiveshell.py:3558: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [1]:
from PyQt5.QtWidgets import QApplication, QPushButton, QVBoxLayout, QWidget
from PyQt5.QtCore import QPropertyAnimation, QTimer, Qt, QRectF, pyqtProperty
from PyQt5.QtGui import QPainter, QColor, QPen
import sys
import math

class RotatingBorderButton(QPushButton):
    def __init__(self, text):
        super().__init__(text)
        self.setFixedSize(200, 80)
        
        # 기본 스타일 - 배경색만 설정하고 테두리는 직접 그릴 것임
        self.setStyleSheet("""
            QPushButton {
                background-color: #000000;
                color: white;
                border: none;  /* 기본 테두리 제거 */
                border-radius: 10px;
                font-size: 16px;
                font-weight: bold;
            }
            QPushButton:hover {
                background-color: #3E3E42;
            }
        """)
        
        # 애니메이션 각도 초기값
        self._angle = 0
        
        # 애니메이션 설정
        self.animation = QPropertyAnimation(self, b"angle")
        self.animation.setDuration(2000)  # 2초 동안 한 바퀴
        self.animation.setStartValue(0)
        self.animation.setEndValue(360)
        self.animation.setLoopCount(-1)  # 무한 반복
        
        # 그라데이션 색상
        self.border_colors = [
            QColor(65, 105, 225),   # 로얄 블루
            QColor(0,0,0),   # 블루바이올렛
            QColor(255, 20, 147),   # 딥 핑크
            QColor(0, 0, 0),  # 핫 핑크
            QColor(255, 69, 0),     # 오렌지 레드
            QColor(255, 215, 0),    # 골드
            QColor(50, 205, 50),    # 라임 그린
            QColor(32, 178, 170)    # 라이트 씨 그린
        ]
        
        # 애니메이션 시작
        self.animation.start()
    
    # 각도 프로퍼티 설정
    def get_angle(self):
        return self._angle
        
    def set_angle(self, angle):
        self._angle = angle
        self.update()  # 화면 갱신
        
    angle = pyqtProperty(float, get_angle, set_angle)
    
    def paintEvent(self, event):
        super().paintEvent(event)
        
        painter = QPainter(self)
        painter.setRenderHint(QPainter.Antialiasing)
        
        # 버튼 영역 설정
        rect = QRectF(5, 5, self.width() - 10, self.height() - 10)
        
        # 8가지 색상으로 분할하여 그리기
        segments = len(self.border_colors)
        angle_per_segment = 360 / segments
        
        for i in range(segments):
            # 현재 세그먼트의 시작과 끝 각도 계산
            start_angle = (self._angle + i * angle_per_segment) % 360
            end_angle = (self._angle + (i + 1) * angle_per_segment) % 360
            
            # QPainter 각도는 16분의 1도 단위임
            start_angle_16 = start_angle * 16
            span_angle_16 = angle_per_segment * 16
            
            # 펜 설정
            pen = QPen(self.border_colors[i])
            pen.setWidth(3)
            painter.setPen(pen)
            
            # 원호 그리기
            painter.drawArc(rect, start_angle_16, span_angle_16)

class Window(QWidget):
    def __init__(self):
        super().__init__()
        self.setWindowTitle("회전하는 외곽선 버튼")
        self.setGeometry(100, 100, 300, 200)
        self.setStyleSheet("background-color: #1E1E1E;")
        
        layout = QVBoxLayout()
        self.setLayout(layout)
        
        rotating_btn = RotatingBorderButton("회전하는 외곽선")
        layout.addWidget(rotating_btn)

if __name__ == "__main__":
    app = QApplication(sys.argv)
    window = Window()
    window.show()
    sys.exit(app.exec_())

C:\Users\kdt\AppData\Local\Temp\ipykernel_7096\2030310172.py:7: DeprecationWarning: sipPyTypeDict() is deprecated, the extension module should use sipPyTypeDictRef() instead
  class RotatingBorderButton(QPushButton):
C:\Users\kdt\AppData\Local\Temp\ipykernel_7096\2030310172.py:92: DeprecationWarning: sipPyTypeDict() is deprecated, the extension module should use sipPyTypeDictRef() instead
  class Window(QWidget):
C:\Users\kdt\AppData\Local\Temp\ipykernel_7096\2030310172.py:90: DeprecationWarning: an integer is required (got type float).  Implicit conversion to integers using __int__ is deprecated, and may be removed in a future version of Python.
  painter.drawArc(rect, start_angle_16, span_angle_16)


SystemExit: 0

c:\Users\kdt\anaconda3\envs\DL_TORCH\lib\site-packages\IPython\core\interactiveshell.py:3558: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [1]:
from PyQt5.QtWidgets import QApplication, QPushButton, QVBoxLayout, QWidget
from PyQt5.QtCore import QPropertyAnimation, QTimer, Qt, QRectF, pyqtProperty, QPointF
from PyQt5.QtGui import QPainter, QColor, QPen, QLinearGradient, QConicalGradient, QRadialGradient
import sys

class NeonBorderButton(QPushButton):
    def __init__(self, text):
        super().__init__(text)
        self.setFixedSize(200, 80)
        
        # 기본 스타일
        self.setStyleSheet("""
            QPushButton {
                background-color: #111111;
                color: #00ffff;
                border: none;
                border-radius: 10px;
                font-size: 16px;
                font-weight: bold;
            }
            QPushButton:hover {
                color: #ffffff;
            }
        """)
        
        # 애니메이션 각도 초기값
        self._angle = 0
        self._glow = 0
        
        # 회전 애니메이션 설정
        self.rotation_animation = QPropertyAnimation(self, b"angle")
        self.rotation_animation.setDuration(6000)  # 6초 동안 한 바퀴
        self.rotation_animation.setStartValue(0)
        self.rotation_animation.setEndValue(360)
        self.rotation_animation.setLoopCount(-1)  # 무한 반복
        
        # 발광 애니메이션 설정
        self.glow_animation = QPropertyAnimation(self, b"glow")
        self.glow_animation.setDuration(1500)  # 1.5초
        self.glow_animation.setStartValue(0)
        self.glow_animation.setEndValue(100)
        self.glow_animation.setLoopCount(-1)  # 무한 반복
        self.glow_animation.setDirection(QPropertyAnimation.Backward)  # 왕복
        
        # 애니메이션 시작
        self.rotation_animation.start()
        self.glow_animation.start()
        
        # 마우스 호버 상태
        self.hovered = False
    
    # 각도 프로퍼티 설정
    def get_angle(self):
        return self._angle
        
    def set_angle(self, angle):
        self._angle = angle
        self.update()
        
    angle = pyqtProperty(float, get_angle, set_angle)
    
    # 발광 프로퍼티 설정
    def get_glow(self):
        return self._glow
        
    def set_glow(self, glow):
        self._glow = glow
        self.update()
        
    glow = pyqtProperty(float, get_glow, set_glow)
    
    def enterEvent(self, event):
        self.hovered = True
        self.update()
        super().enterEvent(event)
        
    def leaveEvent(self, event):
        self.hovered = False
        self.update()
        super().leaveEvent(event)
    
    def paintEvent(self, event):
        super().paintEvent(event)
        
        painter = QPainter(self)
        painter.setRenderHint(QPainter.Antialiasing)
        
        # 버튼 크기
        w, h = self.width(), self.height()
        
        # 네온 효과를 위한 색상
        neon_color = QColor(0, 255, 255, 180 + int(self._glow * 0.75))  # 청록색(시안) 네온
        
        # 외부 발광 효과
        for i in range(3):
            glow_pen = QPen(QColor(0, 255, 255, 40 - i * 10))
            glow_pen.setWidth(3 + i * 2)
            painter.setPen(glow_pen)
            glow_rect = QRectF(4 - i, 4 - i, w - 8 + i * 2, h - 8 + i * 2)
            painter.drawRoundedRect(glow_rect, 10, 10)
        
        # 원뿔형 그라데이션 생성
        gradient = QConicalGradient(QPointF(w/2, h/2), self._angle)
        
        # 그라데이션 색상 설정 - 네온 효과
        gradient.setColorAt(0.0, QColor(0, 255, 255, 255))    # 시안
        gradient.setColorAt(0.25, QColor(0, 200, 255, 200))   # 밝은 파랑
        gradient.setColorAt(0.5, QColor(0, 150, 255, 150))    # 파랑
        gradient.setColorAt(0.75, QColor(0, 200, 255, 200))   # 밝은 파랑
        gradient.setColorAt(1.0, QColor(0, 255, 255, 255))    # 시안
        
        # 펜 설정
        pen = QPen()
        pen.setWidth(2)
        pen.setBrush(gradient)
        painter.setPen(pen)
        
        # 경로 그리기 - 둥근 모서리 사각형
        border_rect = QRectF(4, 4, w - 8, h - 8)
        painter.drawRoundedRect(border_rect, 10, 10)
        
        # 호버 효과 - 내부에 더 밝은 테두리 추가
        if self.hovered:
            inner_pen = QPen(QColor(0, 255, 255, 150 + int(self._glow * 1.05)))
            inner_pen.setWidth(2)
            painter.setPen(inner_pen)
            inner_rect = QRectF(7, 7, w - 14, h - 14)
            painter.drawRoundedRect(inner_rect, 8, 8)

class Window(QWidget):
    def __init__(self):
        super().__init__()
        self.setWindowTitle("네온 테두리 버튼")
        self.setGeometry(100, 100, 300, 200)
        self.setStyleSheet("background-color: #121212;")
        
        layout = QVBoxLayout()
        self.setLayout(layout)
        
        neon_btn = NeonBorderButton("네온 테두리")
        layout.addWidget(neon_btn)

if __name__ == "__main__":
    app = QApplication(sys.argv)
    window = Window()
    window.show()
    sys.exit(app.exec_())

C:\Users\kdt\AppData\Local\Temp\ipykernel_15064\1127228758.py:6: DeprecationWarning: sipPyTypeDict() is deprecated, the extension module should use sipPyTypeDictRef() instead
  class NeonBorderButton(QPushButton):
C:\Users\kdt\AppData\Local\Temp\ipykernel_15064\1127228758.py:130: DeprecationWarning: sipPyTypeDict() is deprecated, the extension module should use sipPyTypeDictRef() instead
  class Window(QWidget):


SystemExit: 0

c:\Users\kdt\anaconda3\envs\DL_TORCH\lib\site-packages\IPython\core\interactiveshell.py:3558: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [1]:
from PyQt5.QtWidgets import QApplication, QPushButton, QVBoxLayout, QWidget
from PyQt5.QtCore import QPropertyAnimation, QSequentialAnimationGroup, pyqtProperty, QRectF
from PyQt5.QtGui import QPainter, QColor, QPen
import sys

class SequentialAnimationButton(QPushButton):
    def __init__(self, text):
        super().__init__(text)
        self.setFixedSize(200, 80)
        
        # 기본 스타일
        self.setStyleSheet("""
            QPushButton {
                background-color: #2D2D30;
                color: white;
                border: none;
                border-radius: 10px;
                font-size: 16px;
                font-weight: bold;
            }
        """)
        
        # 애니메이션 속성들
        self._angle = 0
        self._color_value = 0
        
        # 애니메이션 1: 회전 (0도에서 360도까지)
        self.rotation_anim = QPropertyAnimation(self, b"angle")
        self.rotation_anim.setDuration(2000)  # 2초
        self.rotation_anim.setStartValue(0)
        self.rotation_anim.setEndValue(360)
        
        # 애니메이션 2: 색상 변화 (파란색에서 빨간색으로)
        self.color_anim = QPropertyAnimation(self, b"color_value")
        self.color_anim.setDuration(1500)  # 1.5초
        self.color_anim.setStartValue(0)
        self.color_anim.setEndValue(100)
        
        # 애니메이션 3: 다시 색상 변화 (빨간색에서 파란색으로)
        self.color_back_anim = QPropertyAnimation(self, b"color_value")
        self.color_back_anim.setDuration(1500)  # 1.5초
        self.color_back_anim.setStartValue(100)
        self.color_back_anim.setEndValue(0)
        
        # 순차 애니메이션 그룹 생성
        self.seq_group = QSequentialAnimationGroup()
        self.seq_group.addAnimation(self.rotation_anim)
        self.seq_group.addAnimation(self.color_anim)
        self.seq_group.addAnimation(self.color_back_anim)
        self.seq_group.setLoopCount(-1)  # 무한 반복
        
        # 애니메이션 시작
        self.seq_group.start()
    
    # 각도 프로퍼티
    def get_angle(self):
        return self._angle
        
    def set_angle(self, angle):
        self._angle = angle
        self.update()
        
    angle = pyqtProperty(float, get_angle, set_angle)
    
    # 색상 값 프로퍼티
    def get_color_value(self):
        return self._color_value
        
    def set_color_value(self, value):
        self._color_value = value
        self.update()
        
    color_value = pyqtProperty(float, get_color_value, set_color_value)
    
    def paintEvent(self, event):
        # 기본 버튼 그리기
        super().paintEvent(event)
        
        painter = QPainter(self)
        painter.setRenderHint(QPainter.Antialiasing)
        
        # 버튼 크기
        w, h = self.width(), self.height()
        center_x, center_y = w/2, h/2
        
        # 색상 계산 (파란색에서 빨간색으로 변화)
        blue_component = max(0, 255 - int(2.55 * self._color_value))
        red_component = min(255, int(2.55 * self._color_value))
        border_color = QColor(red_component, 100, blue_component)
        
        # 회전 적용
        painter.translate(center_x, center_y)
        painter.rotate(self._angle)
        painter.translate(-center_x, -center_y)
        
        # 테두리 그리기
        pen = QPen(border_color)
        pen.setWidth(3)
        painter.setPen(pen)
        
        # 약간 작은 사각형 그리기
        border_rect = QRectF(10, 10, w - 20, h - 20)
        painter.drawRoundedRect(border_rect, 8, 8)

class Window(QWidget):
    def __init__(self):
        super().__init__()
        self.setWindowTitle("순차 애니메이션 버튼")
        self.setGeometry(100, 100, 300, 200)
        self.setStyleSheet("background-color: #1E1E1E;")
        
        layout = QVBoxLayout()
        self.setLayout(layout)
        
        seq_btn = SequentialAnimationButton("순차 애니메이션")
        layout.addWidget(seq_btn)

if __name__ == "__main__":
    app = QApplication(sys.argv)
    window = Window()
    window.show()
    sys.exit(app.exec_())

C:\Users\kdt\AppData\Local\Temp\ipykernel_31976\2170707074.py:6: DeprecationWarning: sipPyTypeDict() is deprecated, the extension module should use sipPyTypeDictRef() instead
  class SequentialAnimationButton(QPushButton):
C:\Users\kdt\AppData\Local\Temp\ipykernel_31976\2170707074.py:105: DeprecationWarning: sipPyTypeDict() is deprecated, the extension module should use sipPyTypeDictRef() instead
  class Window(QWidget):


SystemExit: 0

c:\Users\kdt\anaconda3\envs\DL_TORCH\lib\site-packages\IPython\core\interactiveshell.py:3558: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [ ]:

                    item0 = QTableWidgetItem(f"{index}")
                    item0.setTextAlignment(Qt.AlignCenter)
                    self.table.setItem(row, 0, item0)
                    self.table.setColumnWidth(0, 60)
                    
                    desired_width = int(self.total_width * column_ratios[0])
                    final_width = max(desired_width, min_widths[0])
                    self.table.setColumnWidth(0, final_width)
                    
                    
                    
                    
                    item1 = QTableWidgetItem(f"{date[4:6]}/{date[6:8]} - {timestamp[-6:-4]}:{timestamp[-4:-2]}:{timestamp[-2:]}")
                    item1.setTextAlignment(Qt.AlignCenter)
                    self.table.setItem(row, 1, item1)
                    self.table.setColumnWidth(1, 160)

                    item2 = QTableWidgetItem(cam)
                    item2.setTextAlignment(Qt.AlignCenter)
                    self.table.setItem(row, 2, item2)
                    self.table.setColumnWidth(2, 60)

                    item3 = QTableWidgetItem(event)
                    self.table.setItem(row, 3, item3)
                    self.table.setColumnWidth(3, 300)

                    item4 = QTableWidgetItem("▶️")
                    item4.setTextAlignment(Qt.AlignCenter)
                    self.table.setItem(row, 4, item4)
                    self.table.setColumnWidth(4, 50)

                    item5 = QTableWidgetItem(f"{filename}")
                    item5.setTextAlignment(Qt.AlignCenter)
                    self.table.setItem(row, 5, item5)
                    # self.table.setColumnWidth(5, 50)
                # 열 비율과 최소 너비 설정 (전체 6개 열)
column_ratios = [0.08, 0.25, 0.08, 0.35, 0.07, 0.17]  # 총합은 1.0이 되어야 함
min_widths = [60, 160, 60, 300, 50, 100]

# 각 셀에 들어갈 텍스트 포맷 정의
cell_values = [
    f"{index}",
    f"{date[4:6]}/{date[6:8]} - {timestamp[-6:-4]}:{timestamp[-4:-2]}:{timestamp[-2:]}",
    cam,
    event,
    "▶️",
    f"{filename}"
]

# 텍스트 정렬 방식 (None은 정렬 생략)
text_aligns = [
    Qt.AlignCenter,
    Qt.AlignCenter,
    Qt.AlignCenter,
    None,
    Qt.AlignCenter,
    Qt.AlignCenter
]

for col, (value, align) in enumerate(zip(cell_values, text_aligns)):
    item = QTableWidgetItem(value)
    if align is not None:
        item.setTextAlignment(align)
    self.table.setItem(row, col, item)

    # 열 너비 지정
    desired_width = int(self.total_width * column_ratios[col])
    final_width = max(desired_width, min_widths[col])
    self.table.setColumnWidth(col, final_width)
